<a href="https://colab.research.google.com/github/JessicaZezzz/CNN/blob/main/ResNet_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os

In [3]:
#import libraries to look at the images
import matplotlib.pyplot as plt
from PIL import Image

In [4]:
data_src="/content/drive/MyDrive/Dataset_2/"

First we load each image into a list, and save the corresponding label in another list

In [5]:
data=[]
y=[]
for d in os.listdir(data_src):
    for file in os.listdir(data_src+d):
        data.append(Image.open(data_src+d+'/'+file))
        y.append(d)

Convert each image into a numpy array

In [6]:
X=[]
for im in data:
    X.append(np.array(im))

In [7]:
#Convert the list of data into an array
X=np.array(X)

In [8]:
X.shape

(6400, 128, 128)

In [9]:
#Divide by 255 to standardize the values between 0 and 1
X=X/255

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.utils import to_categorical

In [12]:
#Let's count how many of each category we have
non=0
mild=0
mod=0
vm=0
for cat in y:
    if cat=="Mild_Demented":
        mild+=1
    elif cat=="Moderate_Demented":
        mod+=1
    elif cat=="Non_Demented":
        non+=1
    else:
        vm+=1
print("Non Demented: ",non)
print("Very Mild: ",vm)
print("Moderate: ",mod)
print("Mild :",mild)

Non Demented:  3200
Very Mild:  2240
Moderate:  64
Mild : 896


In [13]:
#Now we change these values to numerical ones so that we can use to_categorical
#0 will be non, 1 will be very mild, 2 will be mild, 3 will be moderate
y_num=[]
for cat in y:
    if cat=="Mild_Demented":
        y_num.append(2)
    elif cat=="Moderate_Demented":
        y_num.append(3)
    elif cat=="Non_Demented":
        y_num.append(0)
    else:
        y_num.append(1)

In [14]:
#convert to a categorical target as expected by keras
y=to_categorical(y_num)

In [15]:
y.shape

(6400, 4)

In [16]:
#Reshape to use in a neural network
X=X.reshape(6400,128,128,1)

In [17]:
#Split into a training set and test set
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [18]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(4800, 128, 128, 1) (1600, 128, 128, 1) (4800, 4) (1600, 4)


In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import os 
import tqdm
import glob
import tensorflow 

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import grey2rgb

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [20]:
from tensorflow.keras.applications.vgg16 import VGG16 # VGG16
from tensorflow.keras.applications.vgg19 import VGG19 # VGG19
from tensorflow.keras.applications.resnet50 import ResNet50 # ResNet50
from tensorflow.keras.applications.xception import Xception # Xception
from tensorflow.keras.applications.mobilenet import MobileNet # MobileNet
from tensorflow.keras.applications.nasnet import NASNetMobile # NASNetMobile
from tensorflow.keras.applications.densenet import DenseNet169 # DenseNet169
from tensorflow.keras.applications.densenet import DenseNet121 # DenseNet121
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 # MobileNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.utils import to_categorical
from keras import optimizers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
from typeguard import typechecked
from typing import Optional

In [21]:
import keras
from keras import applications
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from keras import Model
INPUT_SHAPE = (128, 128, 1) # used to define the input size to the model
n_output_units = 4
activation_fn = 'softmax'
resnet_101_model = tf.keras.applications.resnet.ResNet101(include_top=False, weights=None, input_shape=INPUT_SHAPE, pooling='avg')
inputs = Input(shape=INPUT_SHAPE, name='input')
model_base = resnet_101_model(inputs)
output = Dense(units=n_output_units, activation=activation_fn)(model_base)
cnn = Model(inputs=inputs, outputs=output)
cnn.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 128, 128, 1)]     0         
                                                                 
 resnet101 (Functional)      (None, 2048)              42651904  
                                                                 
 dense (Dense)               (None, 4)                 8196      
                                                                 
Total params: 42,660,100
Trainable params: 42,554,756
Non-trainable params: 105,344
_________________________________________________________________


In [22]:
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [52]:
#Train the Convolutional Neural Network on the data
history=cnn.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test))

Epoch 1/20
150/150 [==============================] - 38s 254ms/step - loss: 0.0495 - accuracy: 0.9856 - val_loss: 58.6932 - val_accuracy: 0.4369
Epoch 2/20
150/150 [==============================] - 36s 239ms/step - loss: 0.4855 - accuracy: 0.8210 - val_loss: 385.0113 - val_accuracy: 0.4744
Epoch 3/20
150/150 [==============================] - 38s 251ms/step - loss: 0.4758 - accuracy: 0.8112 - val_loss: 0.4001 - val_accuracy: 0.8363
Epoch 4/20
150/150 [==============================] - 38s 252ms/step - loss: 0.1564 - accuracy: 0.9442 - val_loss: 0.4202 - val_accuracy: 0.8681
Epoch 5/20
150/150 [==============================] - 38s 252ms/step - loss: 0.0765 - accuracy: 0.9725 - val_loss: 0.3072 - val_accuracy: 0.9081
Epoch 6/20
150/150 [==============================] - 38s 252ms/step - loss: 0.0559 - accuracy: 0.9808 - val_loss: 0.1570 - val_accuracy: 0.9594
Epoch 7/20
150/150 [==============================] - 36s 239ms/step - loss: 0.0280 - accuracy: 0.9931 - val_loss: 0.1763 - val

In [53]:
#Get a prediction from the neural network
y_pred=cnn.predict(X_test)

In [54]:
#The prediction is a sequence of 4 probabilities. We choose the label with the highest probability
y_val=[]
for y in y_pred:
    y_val.append(np.argmax(y))

In [55]:
from sklearn.metrics import confusion_matrix,classification_report

In [56]:
#Convert back to labels 0-3
y_true=[]
for y in y_test:
    y_true.append(np.argmax(y))

In [57]:
#Let's see how the predictions did
print(confusion_matrix(y_true,y_val))

[[685  41  45  16]
 [ 28 465  71  16]
 [  4   0 208   2]
 [  0   0   0  19]]


In [58]:
print(classification_report(y_true,y_val))

              precision    recall  f1-score   support

           0       0.96      0.87      0.91       787
           1       0.92      0.80      0.86       580
           2       0.64      0.97      0.77       214
           3       0.36      1.00      0.53        19

    accuracy                           0.86      1600
   macro avg       0.72      0.91      0.77      1600
weighted avg       0.89      0.86      0.87      1600



In [59]:
from sklearn.metrics import accuracy_score

In [60]:
print("Accuracy on test data: ",accuracy_score(y_true,y_val))

Accuracy on test data:  0.860625
